In [48]:
import pandas as pd
import glob

# Preprocessing the raw data

In [49]:
# iterate over all WASDE csv files in the data folder and vertically concatenate them
path = 'data collection/WASDE data/*/*.csv'
all_files = glob.glob(path)
df_from_each_file = (pd.read_csv(f, low_memory=False) for f in all_files)
concatenated_df = pd.concat(df_from_each_file, ignore_index=True)
print(concatenated_df.shape)
concatenated_df.head()

(832923, 16)


,WasdeNumber,ReportDate,ReportTitle,Attribute,ReliabilityProjection,Commodity,Region,MarketYear,ProjEstFlag,AnnualQuarterFlag,Value,Unit,ReleaseDate,ReleaseTime,ForecastYear,ForecastMonth
0,481,April 2010,Mexico Sugar Supply and Use and High Fructose ...,Beginning stocks,NaN,Sugar,Mexico,2008/09,Est.,Annual,1975.0,"1000 Metric Tons, Actual Weight",2010-04-09,08:30:00.0000000,2010,4
1,481,April 2010,Mexico Sugar Supply and Use and High Fructose ...,Domestic,NaN,Sugar,Mexico,2008/09,Est.,Annual,5540.0,"1000 Metric Tons, Actual Weight",2010-04-09,08:30:00.0000000,2010,4
2,481,April 2010,Mexico Sugar Supply and Use and High Fructose ...,Ending Stocks,NaN,Sugar,Mexico,2008/09,Est.,Annual,488.0,"1000 Metric Tons, Actual Weight",2010-04-09,08:30:00.0000000,2010,4
3,481,April 2010,Mexico Sugar Supply and Use and High Fructose ...,Exports,NaN,Sugar,Mexico,2008/09,Est.,Annual,1367.0,"1000 Metric Tons, Actual Weight",2010-04-09,08:30:00.0000000,2010,4
4,481,April 2010,Mexico Sugar Supply and Use and High Fructose ...,Imports,NaN,Sugar,Mexico,2008/09,Est.,Annual,160.0,"1000 Metric Tons, Actual Weight",2010-04-09,08:30:00.0000000,2010,4


In [50]:
crude_oil_df = pd.read_csv('data collection/Crude Oil WTI Futures Historical Data - daily data 10 years.csv')
gold_df = pd.read_csv('data collection/SPDR Gold Shares (GLD) - yahoo finance, 10 years.csv')
sp500_df = pd.read_csv('data collection/S&P 500 Historical Data - daily data 10 years.csv')
silver_df = pd.read_csv('data collection/Silver Futures Historical Data - daily data 10 years.csv')
corn_df = pd.read_csv('data collection/US Corn Futures Historical Data - daily data 10 years.csv')
dollar_rate_df = pd.read_csv('data collection/US Dollar Index Historical Data - daily data 10 years.csv')
soybean_df = pd.read_csv('data collection/US Soybeans Futures Historical Data - daily data 10 years.csv')

cpi_df = pd.read_csv('data collection/Customer Price Index.csv')
cpi_df['DATE'] = pd.to_datetime(cpi_df['DATE'], format='%Y-%m-%d')
cpi_df = cpi_df[cpi_df['DATE'] >= '2013-01']
cpi_df.rename(columns={'CPIAUCSL':'CPI', 'DATE':'Date'}, inplace=True)
ppi_df = pd.read_csv('data collection/Producer Price Index.csv')
ppi_df['DATE'] = pd.to_datetime(ppi_df['DATE'], format='%Y-%m-%d')
ppi_df.rename(columns={'PPIACO':'PPI', 'DATE':'Date'}, inplace=True)
dfs_dict = {
    'crude_oil_df': crude_oil_df,
    'gold_df': gold_df,
    'sp500_df': sp500_df,
    'silver_df': silver_df,
    'dollar_rate_df': dollar_rate_df,
    'cpi_df': cpi_df,
    'ppi_df': ppi_df,
}

In [51]:
def replace_symbols(value):
    if isinstance(value, (int, float, complex)):
        return value
    if isinstance(value, str):
        if '%' in value:
            return float(value.replace('%', ''))
        elif 'K' in value:
            return float(value.replace('K', '')) * 1000
        elif 'M' in value:
            return float(value.replace('M', '')) * 1000000
        elif ',' in value:
            return float(value.replace(',', ''))
    return value

for df_name, df in dfs_dict.items():
    try:
        # Convert all columns except 'Date' using replace_symbols
        for col in df.columns:
            if col != 'Date':
                df[col] = df[col].apply(replace_symbols)
    
    except Exception as e:
        print(f"Error with {df_name}: {e}")
        continue

In [52]:
for df in dfs_dict.values():
    df['Date'] = pd.to_datetime(df['Date'])

# Rename columns to include dataframe name as prefix
for name, df in dfs_dict.items():
    prefix = name.replace('_df', '')  # Remove '_df' from dataframe name
    df.rename(columns=lambda x: f"{prefix}_{x}" if x != 'Date' else x, inplace=True)

# Start with the first dataframe
merged_df = list(dfs_dict.values())[0]
print(merged_df)
# Iteratively merge remaining dataframes
for df in list(dfs_dict.values())[1:]:
    merged_df = pd.merge(merged_df, df, on='Date', how='outer')
    
# Ensure the 'Date' column is in datetime format
merged_df['Date'] = pd.to_datetime(merged_df['Date'])
merged_df = merged_df.sort_values(by='Date', ascending=True)


           Date  crude_oil_Price  crude_oil_Open  crude_oil_High   
0    2024-07-29            75.56           77.28           77.69  \
1    2024-07-28            77.27           77.47           77.48   
2    2024-07-26            77.16           78.35           78.60   
3    2024-07-25            78.28           77.50           78.47   
4    2024-07-24            77.59           77.45           78.19   
...         ...              ...             ...             ...   
3039 2013-01-08            93.15           93.32           93.80   
3040 2013-01-07            93.19           93.21           93.35   
3041 2013-01-04            93.09           92.86           93.21   
3042 2013-01-03            92.92           92.91           93.30   
3043 2013-01-02            93.12           91.78           93.87   

      crude_oil_Low  crude_oil_Vol.  crude_oil_Change %  
0             75.52             NaN               -2.21  
1             77.22             NaN                0.14  
2        

In [53]:
# Filter the DataFrame from 2014 onwards
merged_df = merged_df[merged_df['Date'].dt.year >= 2014]
cleaned_df = merged_df.dropna(axis=1, how='all')
# Display the filtered DataFrame
print(merged_df)

           Date  crude_oil_Price  crude_oil_Open  crude_oil_High   
2787 2014-01-01            98.70           98.61           98.76  \
2786 2014-01-02            95.44           98.50           98.97   
2785 2014-01-03            93.96           95.47           95.74   
2784 2014-01-06            93.43           94.18           94.59   
2783 2014-01-07            93.67           93.60           94.22   
...         ...              ...             ...             ...   
3063 2024-08-26              NaN             NaN             NaN   
3064 2024-08-27              NaN             NaN             NaN   
3065 2024-08-28              NaN             NaN             NaN   
3066 2024-08-29              NaN             NaN             NaN   
3067 2024-08-30              NaN             NaN             NaN   

      crude_oil_Low  crude_oil_Vol.  crude_oil_Change %   gold_Open   
2787          98.54             NaN                0.28         NaN  \
2786          95.34        251250.0      

In [54]:
# Filling NaNs of Sundays (there are Nans on Sundays in all colums except of soybean prices)
sunday_nan_columns = [col for col in cleaned_df.columns if not col.startswith('soy')]
sunday_nan_columns.remove('Date')
cleaned_df[sunday_nan_columns] = cleaned_df[sunday_nan_columns].fillna(method='ffill')
cleaned_df['is_holiday'] = 0
# Update 'is_holiday' to 1 where 'soybean_Price' is NaN
cleaned_df.loc[cleaned_df['gold_Close'].isna(), 'is_holiday'] = 1
# Filling NaNs of Sundays (there are Nans on Sundays in all colums except of soybean prices)
sunday_nan_columns = [col for col in cleaned_df.columns if not col.startswith('soy')]
sunday_nan_columns.remove('Date')
cleaned_df[sunday_nan_columns] = cleaned_df[sunday_nan_columns].fillna(method='ffill')
cleaned_df['is_holiday'] = 0
# Update 'is_holiday' to 1 where 'soybean_Price' is NaN
cleaned_df.loc[cleaned_df['gold_Close'].isna(), 'is_holiday'] = 1
all_data_df = cleaned_df
all_data_df.to_csv('data collection/all_data.csv', index=False)

C:\Users\danam\AppData\Local\Temp\ipykernel_35696\2790439231.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df[sunday_nan_columns] = cleaned_df[sunday_nan_columns].fillna(method='ffill')
C:\Users\danam\AppData\Local\Temp\ipykernel_35696\2790439231.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['is_holiday'] = 0
C:\Users\danam\AppData\Local\Temp\ipykernel_35696\2790439231.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

In [55]:
all_data_df['gold_price_to_predict'] = all_data_df['gold_Close'].shift(-1)
all_data_df['gold_price_yesterday'] = all_data_df['gold_Close'].shift(1)
all_data_df['gold_price_2days'] = all_data_df['gold_Close'].shift(2)
all_data_df['gold_price_3days'] = all_data_df['gold_Close'].shift(3)

all_data_df.to_csv('all_data.csv', index=False)

In [56]:
all_data_df = pd.read_csv('all_data.csv')
all_data_df

,Date,crude_oil_Price,crude_oil_Open,crude_oil_High,crude_oil_Low,crude_oil_Vol.,crude_oil_Change %,gold_Open,gold_High,gold_Low,...,dollar_rate_High,dollar_rate_Low,dollar_rate_Change %,cpi_CPI,ppi_PPI,is_holiday,gold_price_to_predict,gold_price_yesterday,gold_price_2days,gold_price_3days
0,2014-01-01,98.70,98.61,98.76,98.54,NaN,0.28,NaN,NaN,NaN,...,80.24,80.10,0.00,235.288,203.800,1,118.000000,NaN,NaN,NaN
1,2014-01-02,95.44,98.50,98.97,95.34,251250.0,-3.30,117.930000,118.730003,117.750000,...,80.71,80.08,0.75,235.288,203.800,0,119.290001,NaN,NaN,NaN
2,2014-01-03,93.96,95.47,95.74,93.86,215520.0,-1.55,118.639999,119.620003,118.589996,...,80.89,80.49,0.20,235.288,203.800,0,119.500000,118.000000,NaN,NaN
3,2014-01-06,93.43,94.18,94.59,93.20,189380.0,-0.56,119.760002,120.389999,117.110001,...,80.91,80.54,-0.17,235.288,203.800,0,118.820000,119.290001,118.000000,NaN
4,2014-01-07,93.67,93.60,94.22,93.35,177180.0,0.26,118.459999,118.919998,118.129997,...,80.95,80.60,0.22,235.288,203.800,0,118.120003,119.500000,119.290001,118.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2863,2024-08-26,75.56,77.28,77.69,75.52,365120.0,-2.21,233.449997,233.500000,232.059998,...,104.67,104.14,0.27,313.049,255.419,0,233.389999,232.020004,229.369995,232.149994
2864,2024-08-27,75.56,77.28,77.69,75.52,365120.0,-2.21,231.690002,233.470001,231.570007,...,104.67,104.14,0.27,313.049,255.419,0,231.750000,232.759995,232.020004,229.369995
2865,2024-08-28,75.56,77.28,77.69,75.52,365120.0,-2.21,232.000000,232.020004,230.729996,...,104.67,104.14,0.27,313.049,255.419,0,232.949997,233.389999,232.759995,232.020004
2866,2024-08-29,75.56,77.28,77.69,75.52,365120.0,-2.21,231.949997,233.610001,231.919998,...,104.67,104.14,0.27,313.049,255.419,0,231.289993,231.750000,233.389999,232.759995
